In [ ]:
%pip install spotipy

In [ ]:
from spotipy.oauth2 import SpotifyOAuth

scope = 'playlist-modify-public'

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope , client_id=client_id, client_secret=client_secret, redirect_uri='http://localhost:8888/'))

user_id = spotify.me()['id']
user_id

In [ ]:
playlist_url = 'https://open.spotify.com/playlist/7p1D9vXCHZrltctiJfV96B?si=2eaae89c52ac4de7'
playlist = spotify.playlist(playlist_url)

In [ ]:
tracks_query = playlist['tracks']['href']
songs = spotify.user_playlist_tracks(user_id, playlist_id=playlist_url, fields=None, limit=100, offset=0, market=None)
playlist_name, playlist_description = playlist['name'], playlist['description']
original_songs = [f"{song['track']['name']} by {', '.join([artist['name'] for artist in song['track']['artists']])}" for song in songs['items']]
original_songs_ids = [song['track']['id'] for song in songs['items']]

In [ ]:
# write a chatgpt prompt
print (f"Playlist name: {playlist_name}")
print (f"Playlist description: {playlist_description}")
print ("Songs:")
print (*original_songs, sep='\n')
print ("\n")
print ("Suggest songs to add to this playlist:")

In [ ]:
# parse chatgpt output
output = """
Here are some song suggestions to add to the playlist "🧊":

    Snow by Red Velvet
    Chilly by JBJ95
    Winter Song by Sara Bareilles & Ingrid Michaelson
    Let It Snow by Frank Sinatra
    Winter Wonderland by Ella Fitzgerald
    Snow Angel by Taylor Swift
    Snowflakes by Bazzi
    Winter by Marcus & Martinus
    Cold by James Blunt
    All I Want by Kodaline
    Everything About You by Ugly Kid Joe
    Ice Cream by BLACKPINK, Selena Gomez
    Frozen by Madonna
    Winter Wind by Ed Sheeran

These songs have themes of winter, cold, and snow and would fit well with the playlist's description.
"""

# extract song names and artists into a dictionary
songs = {}
for line in output.splitlines():
    if line.startswith('    '):
        song, artist = line[4:].split(' by ')
        songs[song] = artist
        
# print the songs
print (*[f"{song} by {artist}" for song, artist in songs.items()], sep='\n')



In [ ]:
for song, artist in songs.items():
    print(song, artist)

In [ ]:
# get the spotify ids for the songs
ids = []

for song, artist in songs.items():
    result = spotify.search(q=f"track:{song} artist:{artist}", type='track')
    if result['tracks']['items']:
        ids.append(result['tracks']['items'][0]['id'])
        
print (ids)

In [ ]:
output_playlist_name = f"{playlist_name} (suggested by ChatGPT)"
# spotify.user_playlists(user_id, limit=50, offset=0)
user_playlists =  spotify.current_user_playlists(limit=50, offset=0)
# check if playlist already exists in user's library
if any([playlist['name'] == output_playlist_name for playlist in user_playlists['items']]):
    print ("Playlist already exists")
    output_playlist = [playlist for playlist in user_playlists['items'] if playlist['name'] == output_playlist_name][0]
    # add songs to playlist if they don't already exist
else:
    output_playlist = spotify.user_playlist_create(user_id, output_playlist_name, public=True, description=f"{playlist_description} powered by ChatGPT")

current_song_ids = [song['track']['id'] for song in spotify.user_playlist_tracks(user_id, playlist_id=output_playlist['id'], fields=None, limit=100, offset=0, market=None)['items']]
print(current_song_ids)

# add orginal songs to playlist
songs_to_add = []
# add suggested songs to playlist if they don't already exist in output playlist
songs_to_add += [song_id for song_id in ids if song_id not in current_song_ids]
# add original songs to playlist if they don't already exist in output playlist
songs_to_add += [song_id for song_id in original_songs_ids if song_id not in current_song_ids]





print (songs_to_add)
# spotify.user_playlist_add_tracks(user_id, playlist_id=output_playlist['id'], tracks=ids)


In [ ]:

if len(songs_to_add) > 0:
    spotify.user_playlist_add_tracks(user_id, playlist_id=output_playlist['id'], tracks=songs_to_add)